In [1]:
import os

In [2]:
%pwd

'd:\\iNeuron_Projects\\End_To_End_Chest_Cancer_Classification_Using_MLflow_and_DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\iNeuron_Projects\\End_To_End_Chest_Cancer_Classification_Using_MLflow_and_DVC'

### Make data class:

    - update the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### update the configuration manager in src config

In [6]:
from CnnChestCancerMLflowDVC.constants import *                                 
from CnnChestCancerMLflowDVC.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

### Update the components

In [8]:
import os
import zipfile
import gdown
from CnnChestCancerMLflowDVC import logger
from CnnChestCancerMLflowDVC.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

### Update the pipeline :

In [10]:
try:
    # Assume ConfigurationManager and DataIngestionConfig are defined and set up correctly
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error during data ingestion: {e}")
    raise e

[2024-03-16 01:31:10,548: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-16 01:31:10,553: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-16 01:31:10,562: INFO: common: created directory at: artifacts]
[2024-03-16 01:31:10,568: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-16 01:31:10,571: INFO: 1983665874: Downloading data from https://drive.google.com/file/d/1eB4WPSWYFWgXmkwGteICvvum9eGYHUFe/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1eB4WPSWYFWgXmkwGteICvvum9eGYHUFe
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1eB4WPSWYFWgXmkwGteICvvum9eGYHUFe&confirm=t&uuid=e8d101bf-b990-437f-a6da-4eb339696efa
To: d:\iNeuron_Projects\End_To_End_Chest_Cancer_Classification_Using_MLflow_and_DVC\artifacts\data_ingestion\data.zip
100%|██████████| 43.0M/43.0M [00:57<00:00, 752kB/s]

[2024-03-16 01:32:12,126: INFO: 1983665874: Downloaded data from https://drive.google.com/file/d/1eB4WPSWYFWgXmkwGteICvvum9eGYHUFe/view?usp=sharing into file artifacts/data_ingestion/data.zip]
